In [3]:
import json
from tqdm.notebook import tqdm
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename, askdirectory
from pathlib import Path
from os import listdir
from os.path import isfile, join

In [4]:
Tk().withdraw()
# folder_selected = askdirectory()
folder_selected = 'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Datasets/6. Spotify MyData Technical Log and Extended History'

onlyfiles = [f for f in listdir(folder_selected) if isfile(join(folder_selected, f))]
history_files = []
for file in onlyfiles:
    if 'endsong' in file:
        history_files.append(file)

In [5]:
plays = {}
names = {}
total = 0
errors = 0

for file in history_files:
    f = open(folder_selected + '/' + file, encoding="utf8")
    data = json.load(f)
    for play in data:
        total = total + 1
        if 'spotify_track_uri' not in play or play['spotify_track_uri'] is None:
            errors = errors + 1
            continue
        spotify_id = play['spotify_track_uri']
        if spotify_id not in plays:
            plays[spotify_id] = 0
            names[spotify_id] = play['master_metadata_track_name'] + ' - ' + play['master_metadata_album_artist_name']
        plays[spotify_id] = plays[spotify_id] + 1

print('Total: ' + str(total))
print('Errors: ' + str(errors))
print('Unique: ' + str(len(plays)))

Total: 34917
Errors: 165
Unique: 7841


In [6]:
plays_ordered = dict(sorted(plays.items(), key=lambda item: item[1], reverse=True))
for play in plays_ordered:
#     print(play + ' > ' + names[play] + ': ' + str(plays_ordered[play]))
    pass

# Spotify

In [7]:
%env SPOTIPY_CLIENT_ID=79739c5371df43a4b955af5f24560590
%env SPOTIPY_CLIENT_SECRET=84f4caa9d1154f0696a3290fca96438e
%env SPOTIPY_REDIRECT_URI=http://localhost

env: SPOTIPY_CLIENT_ID=79739c5371df43a4b955af5f24560590
env: SPOTIPY_CLIENT_SECRET=84f4caa9d1154f0696a3290fca96438e
env: SPOTIPY_REDIRECT_URI=http://localhost


In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

### Tracks and Features

In [9]:
 

# def download_tracks(track_id_list):
#     download_tracks = []
#     download_features = []
#     to_download = track_id_list
#     extra = []
#     while(True):
#         count = 0
#         count_extra = 0
#         total = len(to_download)
#         print('\t\tThere is still ' + str(total) + ' to download')
#         if total == 0:
#             break
#         if total > 50:
#             total = 50
#         tracks_return = spotify.tracks(to_download[0:total])
#         for track in tracks_return['tracks']:
#             if track['uri'] in to_download:
#                 to_download.remove(track['uri'])
#                 download_tracks.append(track)
#                 count = count + 1
#             else:
#                 extra.append(track['uri'])
#                 count_extra = count_extra + 1
#         print('\t\t\t+' + str(count) + '(-' + str(count_extra) + ')')
#     print('\t\tThe end!! There were ' + str(len(extra)) + ' random tracks')
#     return download_tracks, download_features

In [10]:
print('Will download ' + str(len(plays)) + ' tracks')

tracks, features = download_tracks(list(plays.keys()))

print('Downloaded ' + str(len(tracks)) + ' tracks and ' + str(len(features)) + ' features')

Will download 7841 tracks


  0%|          | 0/157 [00:00<?, ?it/s]

Downloaded 7841 tracks and 7841 features


In [11]:
# Check
tracks_id = []
for track in tracks:
    tracks_id.append(track['uri'])
    
plays_id = []
for play in plays:
    plays_id.append(play)

In [12]:
print('It was downloaded ' + str(len(set(tracks_id))) + ' tracks out of ' + str(len(plays_id)) + ' tracks')
print('BUT')
print(str(len(list(set(tracks_id) - set(plays_id)))) + ' of those tracks where not on the list')
print('And we still have (?) to download ' + str(len(list(set(plays_id) - set(tracks_id)))) +  ' tracks')
# print('Let\'s try again...')

It was downloaded 7841 tracks out of 7841 tracks
BUT
0 of those tracks where not on the list
And we still have (?) to download 0 tracks


In [ ]:
# to_download = list(set(plays_id) - set(tracks_id))
# tracks_d, features_d = download_tracks(to_download)
# tracks.extend(tracks_d)
# features.extend(features_d)

### Artists

In [13]:
artists_track = {}
for track in tracks:
    for artist in track['artists']:
        artists_track[artist['uri']] = artist['name']
        
print('Will Download ' + str(len(artists_track)) + ' artists')

artists = []
for i in tqdm(range(0, len(artists_track), 50)):
    artists_return = spotify.artists(list(artists_track.keys())[i:i+50])
    artists.extend(artists_return['artists'])
    
print('Downloaded ' + str(len(artists)) + ' artists')

Will Download 5003 artists


  0%|          | 0/101 [00:00<?, ?it/s]

Downloaded 5003 artists


### Albuns

In [15]:
albuns_plays = {}
for track in tracks:
    albuns_plays[track['album']['uri']] = track['album']['name']
    
print('There are ' + str(len(albuns_plays)) + ' albuns to download')
old_count = len(albuns_plays)

print('Will look for albuns in ' + str(len(artists_track)) + ' artists')
# Artists Albuns
for artist in tqdm(artists_track):
    albums_artist = spotify.artist_albums(artist)['items']
    for a in albums_artist:
        albuns_plays[a['uri']] = a['name']

print('Added ' + str(len(albuns_plays) - old_count) + ' more to download')

albums = []

print('Downloading...')
for i in tqdm(range(0, len(albuns_plays), 20)):
    albums_return = spotify.albums(list(albuns_plays.keys())[i:i+20])
    albums.extend(albums_return['albums'])
    print('albums count = ' + str(len(albums)), end='\r')
    
print('Downloaded ' + str(len(albums)) + ' Albums')

There are 5964 albuns to download
Will look for albuns in 5003 artists


  0%|          | 0/5003 [00:00<?, ?it/s]

Added 77413 more to download
Downloading...


  0%|          | 0/4169 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Album Tracks

In [ ]:
album_tracks = {}
for album in albums:
    for track in album['tracks']['items']:
        if track['uri'] not in plays:
            album_tracks[track['uri']] = track['name']

In [ ]:
# Save ids before downloading too much and clear memory
f = open('./Datasets/ids_plays_mid.json', 'w', encoding="utf8")
f.write(json.dumps(plays))
f.close()

f = open('./Datasets/ids_albums_plays_mid.json', 'w', encoding="utf8")
f.write(json.dumps(albuns_plays))
f.close()

f = open('./Datasets/ids_albums_tracks_mid.json', 'w', encoding="utf8")
f.write(json.dumps(album_tracks))
f.close()

f = open('./Datasets/ids_artists_mid.json', 'w', encoding="utf8")
f.write(json.dumps(artists_track))
f.close()

In [ ]:
def write_json_iter(it, fh):
    print("[", file=fh)
    for n, rec in tqdm(enumerate(it)):
        if n > 0:
            print(",", file=fh)
        json.dump(rec, fh)
    print("]", file=fh)

In [ ]:
# Save data as well
# Albuns
with open('./Datasets/spotify_albums_mid.json', 'w', encoding="utf8") as out:
    write_json_iter(albums, out)
    out.close()
    
# Artists
with open('./Datasets/spotify_artists_mid.json', 'w', encoding="utf8") as out:
    write_json_iter(artists, out)
    out.close()

In [ ]:
print('Mid Statistics:')
print('There are ' + str(len(plays)) + ' tracks played')
print(str(len(artists_track)) + ' artists played')
print(str(len(albuns_plays)) + ' albums downloaded')
print('and ' + str(len(album_tracks)) + ' tracks from these albuns')

In [ ]:
print('Will download ' + str(len(album_tracks)) + ' tracks from these albuns. Started with ' + str(len(tracks)))

for i in tqdm(range(0, len(album_tracks), 50)):
    tracks_return = spotify.tracks(list(album_tracks.keys())[i:i+50])
    tracks.extend(tracks_return['tracks'])
    features_return = spotify.audio_features(list(album_tracks.keys())[i:i+50])
    features.extend(features_return)
    print('tracks count = ' + str(len(tracks)), end='\r')
    
print('Downloaded new ' + str(len(album_tracks)) + ' tracks')
print('In Total ' + str(len(tracks)) + ' tracks and ' + str(len(features)) + ' features')

## Save Spotify

In [ ]:
# Albuns
with open('./Datasets/spotify_albums.json', 'w', encoding="utf8") as out:
    write_json_iter(albums, out)
    out.close()
    
print('Albums saved')
    
# Artists
with open('./Datasets/spotify_artists.json', 'w', encoding="utf8") as out:
    write_json_iter(artists, out)
    out.close()
    
print('Artists saved')
    
# Features
with open('./Datasets/spotify_features.json', 'w', encoding="utf8") as out:
    write_json_iter(features, out)
    out.close()
    
print('Features saved')
    
# Tracks
with open('./Datasets/spotify_tracks.json', 'w', encoding="utf8") as out:
    write_json_iter(tracks, out)
    out.close()
    
print('Tracks saved')

In [ ]:
f = open('./Datasets/ids_plays.json', 'w', encoding="utf8")
f.write(json.dumps(plays))
f.close()

f = open('./Datasets/ids_albuns_plays.json', 'w', encoding="utf8")
f.write(json.dumps(albuns_plays))
f.close()

f = open('./Datasets/ids_albums_tracks.json', 'w', encoding="utf8")
f.write(json.dumps(album_tracks))
f.close()

f = open('./Datasets/ids_artists.json', 'w', encoding="utf8")
f.write(json.dumps(artists_track))
f.close()

In [ ]:
print('DONE!')

## Test Save

In [ ]:
# Albums

f = open('./Datasets/spotify_albums.json', 'r', encoding="utf8")
albums = json.load(f)
f.close()

print('Albums file Ok! ' + str(len(albums)) + ' albuns')

# Artists

f = open('./Datasets/spotify_artists.json', 'r', encoding="utf8")
artists = json.load(f)
f.close()

print('Artists file Ok! ' + str(len(artists)) + ' artists')

# Features

f = open('./Datasets/spotify_features.json', 'r', encoding="utf8")
features = json.load(f)
f.close()

print('Features file Ok! ' + str(len(features)) + ' features')

# Tracks

f = open('./Datasets/spotify_tracks.json', 'r', encoding="utf8")
tracks = json.load(f)
f.close()

print('Tracks file Ok! ' + str(len(tracks)) + ' tracks')


print('All systems go!')

In [ ]:
# # Load variables
# tracks_id = []
# for track in tracks:
#     tracks_id.append(track['id'])
    
# tracks_id = list(set(tracks_id))

# f = open('./Datasets/ids_tracks_unique.json', 'w', encoding="utf8")
# f.write(json.dumps(tracks_id))
# f.close()

In [ ]:
# f = open('./Datasets/ids_tracks_unique.json', 'r', encoding="utf8")
# tracks_id = json.load(f)
# f.close()

## Playlists

In [29]:
spotipy.Spotify(auth='BQBN6n4loSriG3euAAqKANdaSuYUFa5xUTebFJU4iTs52JpZhd93Kt8hjeHHiFrgqiozBWjVr6O0ahkdKC1D4TvweaEW2xh0oa0oqQQ87VtwLzWb1kNxZkEDNiDoASl_SBMYlYlgXvY9XM64_OyscceemYFgg27G7QMKy2JoszWIpi7r_xCEs9lAmglozZ2DWe2l9KnzTXrdohXzrnhmFtECPpZfIeDpH2GUvZgFMarXng5bF_nZTm5BkylZk5JkfSCzQH0vcJVEP9-SeS1dvIf84A')

In [31]:
token = util.prompt_for_user_token('12127939275')

Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


Enter the URL you were redirected to: https://accounts.spotify.com/authorize?client_id=79739c5371df43a4b955af5f24560590&response_type=code&redirect_uri=http%3A%2F%2Flocalhost


Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


Enter the URL you were redirected to: https://accounts.spotify.com/authorize?client_id=79739c5371df43a4b955af5f24560590&response_type=code&redirect_uri=http%3A%2F%2Flocalhost


SpotifyOauthError: error: invalid_request, error_description: code must be supplied

In [30]:
playlists = spotify.current_user_playlists()

HTTP Error for GET to https://api.spotify.com/v1/me/playlists returned 401 due to Unauthorized.


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/playlists?limit=50&offset=0:
 Unauthorized., reason: None

In [5]:
playlists

{'href': 'https://api.spotify.com/v1/users/spotify/playlists?offset=0&limit=50',
 'items': [{'collaborative': False,
   'description': 'Post Malone is on top of the Hottest 50!',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M'},
   'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M',
   'id': '37i9dQZF1DXcBWIGoYBM5M',
   'images': [{'height': None,
     'url': 'https://i.scdn.co/image/ab67706f00000003d43cd0e140343d6b75a26709',
     'width': None}],
   'name': "Today's Top Hits",
   'owner': {'display_name': 'Spotify',
    'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
    'href': 'https://api.spotify.com/v1/users/spotify',
    'id': 'spotify',
    'type': 'user',
    'uri': 'spotify:user:spotify'},
   'primary_color': None,
   'public': True,
   'snapshot_id': 'MTYyNjA2MjQwMCwwMDAwMDRjNTAwMDAwMTdhOThkZjJhNmQwMDAwMDE3YTg4NjZhYzVm',
   'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXc